# Assignment Sheet Python
## Submission deadline 1.6.2021 10:00 a.m.

### Exercise 1 - Numerical Integration (8 Points)

Write a Python program that integrates the function
\begin{align}
    f(x,y) = exp(-0.5(x^2+y^2))
\end{align}
in the limits $x \in [-1,1]$ and $y \in [-1,1]$ numerically. To do so, draw a few million random
numbers for each x, y, and z and store these random numbers in arrays. Count, how often
the value of z is less than f(x,y). Choose an appropriate range for z. From the result, estimate
the integral
\begin{align}
    \int_{-1}^1\int_{-1}^1 f(x,y) dx dy
\end{align}

In [ ]:
import math
import time
import numpy as np

# number of samples
N = 10000000

x = np.random.rand(N) * 2 - 1
y = np.random.rand(N) * 2 - 1
z = np.random.rand(N)

def integrate(x, y, z, target=None, tid=0):
    results = np.exp(-0.5 * (x*x + y*y))
    count = np.count_nonzero(z < results)
    if target:
        target[tid] = count
    return count

In [ ]:
# a pure python solution for speed comparison (result: about 10 times slower)
def integrate_pure_python(x, y, z, target=None, tid=0):
    count = 0
    for i in range(len(x)):
        if z[i] < math.exp(-0.5 * (x[i] * x[i] + y[i] * y[i])):
            count += 1
    if target:
        target[tid] = count
    return count

#### a) How long does the integration take without running parallel?

In [ ]:
start = time.perf_counter()
count = integrate(x, y, z)
end = time.perf_counter()

print(f"count: {count}, value of integral: {count / N * 4}\nelapsed time: {end - start:.4f} seconds")

Your Answer Here:

Without parallelization the integration took 0.1061 seconds (mean of 5 runs) for ten million points.

#### b) Rewrite your code using multi-threading. Create 10 threads, each thread processing a certain range of the created points.

In [ ]:
import threading

# setup, note that this code assumes, that N is divisble by num_threads, which may not always be the case
num_threads = 10
threads = []
results = [0] * num_threads
batchsize = N // num_threads
for i in range(num_threads):
    start_index = i * batchsize
    end_index = (i + 1) * batchsize # max batch index + 1, because used in slices
    threads.append(threading.Thread(target=integrate, args=(x[start_index:end_index], y[start_index:end_index], z[start_index:end_index], results, i)))

# execution
start = time.perf_counter()
for t in threads:
    t.start()
for t in threads:
    t.join()
end = time.perf_counter()
print(f"count: {sum(results)}\nelapsed time: {end - start:.4f} seconds")

Your Answer Here:

Parallelizing the code using 10 Threads, the run time dropped to 0.0485 seconds (mean of 5 runs, executed on 4 cores on https://jupyter-cloud.gwdg.de)

#### c) Now, create a single thread for the evaluation of each random point. Compare the performance to the result from b).

In [ ]:
# same code as b, except:
num_threads = N
# probably not "optimal" for this assignment, but I'm to lazy to invest more time in such an awful idea.

Your Answer here:

With ten million points I interrupted the interpreter after two minutes or so. With `N = 1000` it took about 0.11 seconds, which was about 40 times the duration of b).

Don't do this.

#### d) Rewrite your code using multi-processing. Create a pool and use the Pool.apply, Pool.map or Pool.starmap function. Compare the CPU occupancy for multi-processing and multithreading with a tool like top or htop. How many python processes are running in both cases? Which fraction of CPU time are the processes maximally occupying?

In [ ]:
import multiprocessing as mp

# setup
pool_size = mp.cpu_count()
pool = mp.Pool(pool_size)

start = time.perf_counter()
# could use np.array_split() if N % pool_size != 0
result = pool.starmap(integrate, zip(np.split(x, pool_size), np.split(y, pool_size), np.split(z, pool_size)))
end = time.perf_counter()
print(f"count: {sum(results)}\nelapsed time: {end - start:.4f} seconds")
pool.close()

Your Answer here:

In this case the integration took 0.4497 seconds (again, mean of 5 runs). In the `multiprocessing` case, four python processes are running, but only one when using  the `threading` module.

`top` didn't show very meaningful output for the CPUI utilization, because I couldn't get runtimes of more than 1 second for the theading example without crashing the kernel by exausting the available memory. I assume that by using almost only `numpy` functions, which are implemented in C and not Python, the GIL doesn't slow down the execution and the overhead of theads is low compared to full processes. When using the slow pure python implementation, `multiprocessing` is obviously much faster than `threading` (~ 3.7 s to ~ 20 s). With threads, it's even slower than without parallelization.

#### e) Change the previous program to use asynchronous execution of parallel processes. Compare the performance between synchronous and asynchronous execution.

In [ ]:
# setup
pool_size = mp.cpu_count()
pool = mp.Pool(pool_size)

start = time.perf_counter()
# could use np.array_split() if N % pool_size != 0
result = pool.starmap_async(integrate, zip(np.split(x, pool_size), np.split(y, pool_size), np.split(z, pool_size)))
result.wait()
end = time.perf_counter()
print(f"count: {sum(results)}\nelapsed time: {end - start:.4f} seconds")
pool.close()

Your Answer Here:

The average runtime of 5 runs was 0.5270 seconds in the asynchronous example, which is very similar to the synchronous one above (there was some overlap between the measured times for single runs).